In [36]:
import pandas as pd
import copy
import os

In [2]:
df_link = pd.read_csv("ChinaVis_data/Link.csv")
df_link = df_link.drop_duplicates()
df_link.head()

,relation,source,target
0,r_subdomain,Domain_34a6231f101fdfa2b051beaa4b94d463fe5f9f4...,Domain_5052db3f33d5337ab631025f7d5de3c5ac559ed...
1,r_dns_a,Domain_34a6231f101fdfa2b051beaa4b94d463fe5f9f4...,IP_37f7ed5739b43757ff23c712ae4d60d16615c59c081...
2,r_cert,Domain_34a6231f101fdfa2b051beaa4b94d463fe5f9f4...,Cert_9ace6aae20e3ac6d9ebfae8938b91112460b27ad2...
3,r_dns_a,Domain_5052db3f33d5337ab631025f7d5de3c5ac559ed...,IP_37f7ed5739b43757ff23c712ae4d60d16615c59c081...
4,r_dns_a,Domain_32b4d5d93789d5436fe729499c7b4d311742797...,IP_37f7ed5739b43757ff23c712ae4d60d16615c59c081...


In [3]:
df_node = pd.read_csv("ChinaVis_data/Node.csv")
print("#node: %d" % len(df_node))
df_node.head()

#node: 2371558


,id,name,type,industry
0,Domain_34a6231f101fdfa2b051beaa4b94d463fe5f9f4...,34a6231f10.com,Domain,"['C', 'E', 'A', 'B']"
1,Domain_5052db3f33d5337ab631025f7d5de3c5ac559ed...,5052db3f33.com,Domain,"['C', 'E', 'A', 'B']"
2,IP_37f7ed5739b43757ff23c712ae4d60d16615c59c081...,5.180.xxx.xxx,IP,[]
3,Cert_9ace6aae20e3ac6d9ebfae8938b91112460b27ad2...,9ace6aae20,Cert,[]
4,Domain_32b4d5d93789d5436fe729499c7b4d311742797...,32b4d5d937.com,Domain,"['C', 'E', 'A', 'B']"


In [4]:
type_cores = ['Domain', 'IP', 'Cert']
df_cores = df_node[df_node['type'].isin(type_cores)]
print("#node: %d" % len(df_cores))
df_cores.head()

#node: 2339835


,id,name,type,industry
0,Domain_34a6231f101fdfa2b051beaa4b94d463fe5f9f4...,34a6231f10.com,Domain,"['C', 'E', 'A', 'B']"
1,Domain_5052db3f33d5337ab631025f7d5de3c5ac559ed...,5052db3f33.com,Domain,"['C', 'E', 'A', 'B']"
2,IP_37f7ed5739b43757ff23c712ae4d60d16615c59c081...,5.180.xxx.xxx,IP,[]
3,Cert_9ace6aae20e3ac6d9ebfae8938b91112460b27ad2...,9ace6aae20,Cert,[]
4,Domain_32b4d5d93789d5436fe729499c7b4d311742797...,32b4d5d937.com,Domain,"['C', 'E', 'A', 'B']"


### 2022.05.19 Plan: search neighbors with depth 3, record all ip and cert nodes.

In [12]:
df_link['relation'].unique()

array(['r_subdomain', 'r_dns_a', 'r_cert', 'r_asn', 'r_cert_chain',
       'r_whois_name', 'r_whois_email', 'r_whois_phone', 'r_cidr',
       'r_cname', 'r_request_jump'], dtype=object)

In [13]:
very_strong_links = ["r_cert", "r_subdomain", "r_request_jump", "r_dns_a"]
strong_links = ["r_whois_name", "r_whois_email", "r_whois_phone"]
common_links = ["r_cert_chain", "r_cname"]
weak_links = ["r_asn", "r_cidr"]
# ignore_links = ["r_cert", "r_asn", "r_cidr"]

df_link["link_strength"] = 0
df_link.loc[df_link['relation'].isin(very_strong_links), 'link_strength'] = 3
df_link.loc[df_link['relation'].isin(strong_links), 'link_strength'] = 3
df_link.loc[df_link['relation'].isin(common_links), 'link_strength'] = 2
df_link.loc[df_link['relation'].isin(weak_links), 'link_strength'] = 1
# df_link.loc[df_link['relation'].isin(ignore_links), 'link_strength'] = 0
df_link.head()

,relation,source,target,link_strength
0,r_subdomain,Domain_34a6231f101fdfa2b051beaa4b94d463fe5f9f4...,Domain_5052db3f33d5337ab631025f7d5de3c5ac559ed...,3
1,r_dns_a,Domain_34a6231f101fdfa2b051beaa4b94d463fe5f9f4...,IP_37f7ed5739b43757ff23c712ae4d60d16615c59c081...,3
2,r_cert,Domain_34a6231f101fdfa2b051beaa4b94d463fe5f9f4...,Cert_9ace6aae20e3ac6d9ebfae8938b91112460b27ad2...,0
3,r_dns_a,Domain_5052db3f33d5337ab631025f7d5de3c5ac559ed...,IP_37f7ed5739b43757ff23c712ae4d60d16615c59c081...,3
4,r_dns_a,Domain_32b4d5d93789d5436fe729499c7b4d311742797...,IP_37f7ed5739b43757ff23c712ae4d60d16615c59c081...,3


### Explorer

In [ ]:
illegal_domains = df_node[df_node['industry'] != '[]'].to_list()
illegal_domains

In [14]:
cores = set()
visited_nodes = set()


def Search(node_id, depth, path):
    visited_nodes.add(node_id)

    if len(visited_nodes) % 3000 == 0:
        print("#visited nodes: %d\t#core nodes: %d" % (len(visited_nodes), len(cores)))

    if depth == 0: return

    df = df_link[(df_link['source'] == node_id) | (df_link['target'] == node_id)]
    if node_id.startswith("Cert"):
        df =  df[(df['relation'] != 'r_cert') | (df['source'].isin(illegal_domains)) | (df['source'].isin(cores))]


    neighbors = []
    for _, link in df.iterrows():
        if link['source'] not in visited_nodes:
            neighbors.append({"id": link['source'], "depth": link['link_strength']})

        if link['target'] not in visited_nodes:
            neighbors.append({"id": link['target'], "depth": link['link_strength']})

    # find a clue node!
    if (len(df[(~df['source'].isin(path)) & (df['source'].isin(clues))]) != 0) or (len(
            df[(~df['target'].isin(path)) & (df['target'].isin(clues))]) != 0):
        print("Find a path.")
        for path_node in path:
            cores.add(path_node)

    # if len(neighbors) > 50:
    #     print(node_id)
    #     print("#neighbor: %d" % len(neighbors))

    for neighbor in neighbors:
        node_id = neighbor["id"]

        path.append(node_id)
        Search(node_id, min(depth - 1, neighbor["depth"]), path)
        path.remove(node_id)

### Drawer

In [215]:
from pyecharts.charts import Graph
from pyecharts import options as opts


def Draw(nodes, df_links, name):
    name_length = 15

    node_list = []
    for node in nodes:
        if node in clues:
            node_list.append({"name": node[0:name_length], "itemStyle": {"color": "#71995a"}})
        else:
            node_list.append({"name": node[0:name_length]})

    link_list = []
    for index, row in df_links.iterrows():
        link_list.append(
            {"source": row['source'][0:name_length], "target": row['target'][0:name_length], "value": row['relation']})

    print("#node: %d\t#links: %d" % (len(node_list), len(link_list)))

    c = (
        Graph()
            .add("", node_list, link_list, repulsion=1000, edge_label=opts.LabelOpts(
            is_show=True, position="middle", formatter="relation"))
            .set_global_opts(title_opts=opts.TitleOpts(title=name))
            .render(name + ".html")
    )

### JSON Generator

In [216]:
def ToJSON(nodes, df_links, name):
    df_json_node = df_node[df_node['id'].isin(nodes)]
    df_json_node = df_json_node.rename(columns={"type": "group"})

    df_json_link = df_links.drop(['link_strength'], axis=1)
    df_json_link = df_json_link.rename(columns={"relation": "value"})

    if not os.path.exists(name):
        os.mkdir(name)
    df_json_node.to_json(name + "/nodes.json", orient="records")
    df_json_link.to_json(name + "/links.json", orient="records")

In [217]:
def Filter(relation, df_graph, cores, threshold=20):
    df_relation = df_graph[df_graph['relation'] == relation]
    domains = []
    uniques = df_relation['target'].unique()
    for unique in uniques:
        df = df_relation[df_relation['target'] == unique]
        if len(df) > threshold:
            df = df.sample(threshold)
        domains += df['source'].to_list()

    df_graph = df_graph[
        (df_graph['relation'] != relation) | (df_graph['source'].isin(domains)) | df_graph['source'].isin(cores)]

    return df_graph

In [248]:
name = "group-4"
if not os.path.exists(name):
    os.mkdir(name)

# clues = ["Domain_c58c149eec59bb14b0c102a0f303d4c20366926b5c3206555d2937474124beb9",
#          "Domain_f3554b666038baffa5814c319d3053ee2c2eb30d31d0ef509a1a463386b69845"]

# clues = ["Domain_b10f98a9b53806ccd3a5ee45676c7c09366545c5b12aa96955cde3953e7ad058",
#          "IP_400c19e584976ff2a35950659d4d148a3d146f1b71692468132b849b0eb8702c"]
#
# clues = ["Domain_24acfd52f9ceb424d4a2643a832638ce1673b8689fa952d9010dd44949e6b1d9",
#          "Domain_9c72287c3f9bb38cb0186acf37b7054442b75ac32324dfd245aed46a03026de1",
#          "Domain_717aa5778731a1f4d6f0218dd3a27b114c839213b4af781427ac1e22dc9a7dea",
#          "Domain_8748687a61811032f0ed1dcdb57e01efef9983a6d9c236b82997b07477e66177",
#          "Whois_Phone_f4a84443fb72da27731660695dd00877e8ce25b264ec418504fface62cdcbbd7"]
#
# clues = ["IP_7e730b193c2496fc908086e8c44fc2dbbf7766e599fabde86a4bcb6afdaad66e",
#          "Cert_6724539e5c0851f37dcf91b7ac85cb35fcd9f8ba4df0107332c308aa53d63bdb"]
#
clues = ["Whois_Phone_fd0a3f6712ff520edae7e554cb6dfb4bdd2af1e4a97a39ed9357b31b6888b4af",
         "IP_21ce145cae6730a99300bf677b83bbe430cc0ec957047172e73659372f0031b8",
         "Domain_7939d01c5b99c39d2a0f2b418f6060b917804e60c15309811ef4059257c0818a",
         "Domain_587da0bac152713947db682a5443ef639e35f77a3b59e246e8a07c5eccae67e5"]

In [ ]:
cores.clear()
visited_nodes.clear()
for clue in clues:
    cores.add(clue)

for clue in clues:
    Search(clue, 3, [clue])

print("#cores: %d" % len(cores))
print("#visited nodes: %d" % len(visited_nodes))

Find a path.
Find a path.
Find a path.
#visited nodes: 3000	#core nodes: 7
Find a path.
Find a path.
Find a path.
Find a path.
#visited nodes: 6000	#core nodes: 8
#visited nodes: 9000	#core nodes: 8


### Clue Sketch

In [ ]:
sketch_nodes = copy.deepcopy(cores)
df_sketch = df_link[(df_link['source'].isin(sketch_nodes)) & (df_link['target'].isin(sketch_nodes))]

# filter cert and ip
df_sketch = Filter('r_cert', df_sketch, sketch_nodes)
df_sketch = Filter('r_dns_a', df_sketch, sketch_nodes)
df_sketch = Filter('r_whois_phone', df_sketch, sketch_nodes)

# sketch_nodes = df_sketch['source'].to_list() + df_sketch['target'].to_list()
# sketch_nodes = set(sketch_nodes)

Draw(sketch_nodes, df_sketch, name + "/sketch")
ToJSON(sketch_nodes, df_sketch, name + "/sketch")

### Cores Nodes

In [ ]:
# find all related ip and certs along the path
cores_ip_cert = copy.deepcopy(cores)
df_depth = df_link[(df_link['source'].isin(cores)) | (df_link['target'].isin(cores))]
nodes = df_depth['source'].to_list() + df_depth['target'].to_list()
for node in nodes:
    if node.startswith("IP") or node.startswith("Cert"):
        cores_ip_cert.add(node)

df_cores_ip_certs = df_link[(df_link['source'].isin(cores_ip_cert)) & (df_link['target'].isin(cores_ip_cert))]

# filter cert and ip
df_cores_ip_certs = Filter('r_cert', df_cores_ip_certs, cores_ip_cert)
df_cores_ip_certs = Filter('r_dns_a', df_cores_ip_certs, cores_ip_cert)
df_cores_ip_certs = Filter('r_whois_phone', df_cores_ip_certs, cores_ip_cert)

cores_nodes = df_cores_ip_certs['source'].to_list() + df_cores_ip_certs['target'].to_list()
cores_nodes = set(cores_nodes)

Draw(cores_nodes, df_cores_ip_certs, name + "/cores")
ToJSON(cores_nodes, df_cores_ip_certs, name + "/cores")

### Subgraph - all

In [ ]:
subgraph_nodes = copy.deepcopy(cores_nodes)
df_subgraph = df_link[(df_link['source'].isin(subgraph_nodes)) | (df_link['target'].isin(subgraph_nodes))]

# filter cert and ip
df_subgraph = Filter('r_cert', df_subgraph, subgraph_nodes)
df_subgraph = Filter('r_dns_a', df_subgraph, subgraph_nodes)

subgraph_nodes = df_subgraph['source'].to_list() + df_subgraph['target'].to_list()
subgraph_nodes = set(subgraph_nodes)

Draw(subgraph_nodes, df_subgraph, name + "/subgraph-all")
ToJSON(subgraph_nodes, df_subgraph, name + "/subgraph-all")

### Subgraph - illegal

In [ ]:
# if we only care illegal nodes?
illegal_domain = df_node[df_node['industry'] != '[]']['id']
df_subgraph_illegal = df_subgraph[
    (df_subgraph['source'].isin(illegal_domain)) | df_subgraph['source'].isin(cores_nodes)]

subgraph_nodes_illegal = df_subgraph_illegal['source'].to_list() + df_subgraph_illegal['target'].to_list()
subgraph_nodes_illegal = set(subgraph_nodes_illegal)

Draw(subgraph_nodes_illegal, df_subgraph_illegal, name + "/subgraph-illegal")
ToJSON(subgraph_nodes_illegal, df_subgraph_illegal, name + "/subgraph-illegal")